In [113]:
# !pip install langchain_chroma
# ! pip install sentence_transformers
# ! pip install faiss-cpu
! pip install -q groq



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


A RAG system is a system that help our llm to generalize in a specifique content or local content(private).
building a rag system require the steps:

1. devide the document into chunks to suit the llm context window
2. embedd the chunks and then store them in a vector db
3. after the query arrive, embedd it then retreive k-chunks that has similarity with the query
4. finaly include the k-chunks into the prompt

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Preparing the data, pdf file

In [51]:
loader = PyPDFLoader("./loi-n-01-00-portant-organisation-de-lenseignement-supérieur.pdf")
pages = loader.load_and_split()

In [52]:
pages[5].page_content

"président est prépondérante. \n \nArticle 12 \nLe conseil de l'université délibère sur toutes les questions relatives aux missions et à la bonne marche de l'université. \nA cet effet, et outre les attributions qui lui sont dévolues par la présente loi, il :"

the second part is to devide the text to chunks, we will use the langchaine text splitter for this

In [53]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
splits = text_splitter.split_documents(pages)

In [63]:
type(splits[1])

langchain_core.documents.base.Document

In [55]:
len(splits)

109

As the embedding models from langchain require api_key, which may cost me something, i will do an alternative which is using a free embedding model from hugginface as they are open source

because the suitable format for huggin face model are plaint text, i transformed the splited text which is of type -langchain_core.documents.base.Document-, to simple text chuncks

In [ ]:
text_chunks = [doc.page_content for doc in splits]

In [ ]:
text_chunks[0]

"20 \nuniversités prévu à l'article 17 ci-dessus, les personnels de l' Etat sus-mentionnés demeurent ré gis par les statuts particulier s \ndont ils relèvent. \n \nArticle 91 \nLa situation conférée par le statut des personnels des universités aux personnels transférés en vertu de l'article 90 ci-dessus ne saurait, en \naucun cas, être moins favorable que leur situation statutaire à la date de leur transfert. \n \nArticle 92 \nLes services effectués par les personnels visés à l'article 90 dans les universités, dans les établissements universitaires et dans \nl'administration sont considérés comme ayant été effectués au sein des universités. \n \nArticle 93 \nNonobstant toutes dispositions contraires, les personnels transférés ou intégrés aux universités en application des dispositions"

# 2. Embedding generation and store them to the vector DB

for this task i will be using Aleph Alpha's semantic embeddings from the langchain embedding models 

In [62]:
from sentence_transformers import SentenceTransformer

c:\Users\dell\Desktop\MS_DS\M1\S2\AppAutomaique\Projet\loi001_project\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [69]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [70]:
embeddings = [model.encode(chunk) for chunk in text_chunks]

In [83]:
import numpy as np

In [ ]:
embedding_array = np.array(embeddings) #to store it into faiss

In [90]:
embedding_array.shape

(109, 384)

Store the embedding into a vector DB using FAISS

In [93]:
import faiss

In [94]:
embedding_dim = embedding_array.shape[1]  # Dimension of your embeddings
index = faiss.IndexFlatL2(embedding_dim)

In [95]:
index.add(embedding_array)

## 3. Retrieving the chuncks with most similarity

In [ ]:
def retrieving_chunks(query):
    # Example query
    # query = "Quelles sont les prestations et le financement des services sociaux destinés aux étudiants dans le cadre de la vie universitaire ?"
    query_embedding = model.encode(query)  # Generate embedding for the query

    # Convert to NumPy array and reshape for FAISS
    query_embedding = np.array([query_embedding], dtype=np.float32)

    # Perform the search
    k = 3  # Number of nearest neighbors to retrieve
    distances, indices = index.search(query_embedding, k)
    matching_chunks = [text_chunks[i] for i in indices[0]]
    return '/n'.join(matching_chunks)

## 3. Generating Step

In [143]:
import json

with open('config.json') as config_file:
    config = json.load(config_file)
    
api_key = config['api_key']

In [144]:
from groq import Groq

client = Groq(
    api_key=api_key,
)

In [139]:
def llm_generation(question, context):
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": "repondre au question en se basant sur le context donnée"
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": "question : "+question+",context: "+context
            }
        ],
        model="llama3-70b-8192",
    )

    print(chat_completion.choices[0].message.content)

In [142]:
respoons = llm_generation('Quelles sont les prestations et le financement des services sociaux destinés aux étudiants dans le cadre de la vie universitaire ?',ret)

Selon le contexte donné, les prestations des services sociaux destinés aux étudiants dans le cadre de la vie universitaire comprennent :

* L'hébergement
* La restauration
* La couverture sanitaire
* Les bourses et prêts d'études
* Un système de bourses destiné aux étudiants méritants démunis

Le financement de ces services sociaux est assuré par :

* Des subventions de l'Etat
* Des collectivités locales
* Des établissements d'enseignement supérieur
* La participation des bénéficiaires
* Des contributions volontaires de personnes physiques ou morales.
